### Import des données dans minio
Penser a copier data dans le notebook :
`docker cp data jupyter:/data`

In [3]:
!ls /data

flight_sample_2022-09-01.csv  flight_sample_2022-09-16.csv
flight_sample_2022-09-02.csv  flight_sample_2022-09-17.csv
flight_sample_2022-09-03.csv  flight_sample_2022-09-18.csv
flight_sample_2022-09-04.csv  flight_sample_2022-09-19.csv
flight_sample_2022-09-05.csv  flight_sample_2022-09-20.csv
flight_sample_2022-09-06.csv  flight_sample_2022-09-21.csv
flight_sample_2022-09-07.csv  flight_sample_2022-09-22.csv
flight_sample_2022-09-08.csv  flight_sample_2022-09-23.csv
flight_sample_2022-09-09.csv  flight_sample_2022-09-24.csv
flight_sample_2022-09-10.csv  flight_sample_2022-09-25.csv
flight_sample_2022-09-11.csv  flight_sample_2022-09-26.csv
flight_sample_2022-09-12.csv  flight_sample_2022-09-27.csv
flight_sample_2022-09-13.csv  flight_sample_2022-09-28.csv
flight_sample_2022-09-14.csv  flight_sample_2022-09-29.csv
flight_sample_2022-09-15.csv  flight_sample_2022-09-30.csv


In [4]:
from minio import Minio

minio_ip_address = "minio"

client_minio = Minio(
    f"{minio_ip_address}:9000",
    access_key="root",
    secret_key="password",
    secure=False
)

# Création du bucket tp6
if client_minio.bucket_exists("projet") == False:
    client_minio.make_bucket("projet")



In [5]:
import os

# Importer les données dans minio
for root, dirs, files in os.walk("/data"):
    for file in files:
        local_file_path = os.path.join(root, file)
        minio_file_path = os.path.join("data", file)
        client_minio.fput_object("projet", minio_file_path, local_file_path)

# Afficher les fichiers présents dans minio
objects = client_minio.list_objects('projet', prefix='data/', recursive=True)

for obj in objects:
    print(obj.object_name)

data/flight_sample_2022-09-01.csv
data/flight_sample_2022-09-02.csv
data/flight_sample_2022-09-03.csv
data/flight_sample_2022-09-04.csv
data/flight_sample_2022-09-05.csv
data/flight_sample_2022-09-06.csv
data/flight_sample_2022-09-07.csv
data/flight_sample_2022-09-08.csv
data/flight_sample_2022-09-09.csv
data/flight_sample_2022-09-10.csv
data/flight_sample_2022-09-11.csv
data/flight_sample_2022-09-12.csv
data/flight_sample_2022-09-13.csv
data/flight_sample_2022-09-14.csv
data/flight_sample_2022-09-15.csv
data/flight_sample_2022-09-16.csv
data/flight_sample_2022-09-17.csv
data/flight_sample_2022-09-18.csv
data/flight_sample_2022-09-19.csv
data/flight_sample_2022-09-20.csv
data/flight_sample_2022-09-21.csv
data/flight_sample_2022-09-22.csv
data/flight_sample_2022-09-23.csv
data/flight_sample_2022-09-24.csv
data/flight_sample_2022-09-25.csv
data/flight_sample_2022-09-26.csv
data/flight_sample_2022-09-27.csv
data/flight_sample_2022-09-28.csv
data/flight_sample_2022-09-29.csv
data/flight_sa

### Effectuer une requête à l'aide de SparkSQL

In [6]:
### Configuration Mac : utilisation de notebook Jupyter

from pyspark import SparkContext, SparkConf

conf = SparkConf() \
    .setAppName('SparkApp') \
    .setMaster('spark://spark:7077') \
    .set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .set("spark.sql.shuffle.partitions", "10")


sc = SparkContext.getOrCreate(conf=conf)

from pyspark.sql import SQLContext
# Créer un SQLContext pour les opérations SQL
sql_context = SQLContext(sc)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dd83fd72-57eb-41c1-b563-f85d9b4ee854;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.3.4!hadoop-aws.jar (587ms)
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.262/aws-java-sdk-bundle-1.12.262.jar ...
	[SUCCESSFUL ] com.amazonaws#aws-java-sdk-bundle;1.12.262!aws-java-sdk-bundle.jar (47774ms)
downloading https://repo1.maven.org/maven2/org/wildfly/openssl/wildfly-openssl/1.0.7.Final/wildfly-openssl-1.0.7.Final.jar ...
	[SUCCESSFUL ]

In [9]:
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", f"http://{minio_ip_address}:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "root")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "password")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")


In [19]:
from pyspark.sql.functions import explode, split, col

# Définir la source (le répertoire "data" contenant les fichiers csv)
input_dir = "s3a://projet/data"  # Répertoire de surveillance

# Lire les fichiers CSV en tant que DataFrame statique
df = sql_context.read.format("csv").option("header", "true").load(input_dir)


In [20]:
# Enregistrer le DataFrame en tant que table temporaire
df.createOrReplaceTempView("flights")

# Top 5 de la correspondance la plus fréquente
query = """
SELECT airportofdeparture, airportofdestination, COUNT(*) as count
FROM flights
WHERE airportofdeparture IS NOT NULL AND airportofdestination IS NOT NULL
GROUP BY airportofdeparture, airportofdestination
ORDER BY count DESC
LIMIT 5
"""

top_5_couples = sql_context.sql(query)

# Afficher les résultats
top_5_couples.show()

+------------------+--------------------+-----+
|airportofdeparture|airportofdestination|count|
+------------------+--------------------+-----+
|              LFBO|                LFPO|  542|
|              LEPA|                LEIB|  455|
|              ENVA|                ENGM|  391|
|              EDDB|                EDDF|  366|
|              ENZV|                ENGM|  356|
+------------------+--------------------+-----+



In [23]:
# Enregistrer le DataFrame en tant que table temporaire
df.createOrReplaceTempView("flights")

# Top 5 de la route la plus fréquente
query = """
SELECT 
    CASE 
        WHEN airportofdeparture < airportofdestination THEN airportofdeparture 
        ELSE airportofdestination 
    END as airport1,
    CASE 
        WHEN airportofdeparture < airportofdestination THEN airportofdestination 
        ELSE airportofdeparture 
    END as airport2,
    COUNT(*) as count
FROM flights
WHERE airportofdeparture IS NOT NULL AND airportofdestination IS NOT NULL
GROUP BY airport1, airport2
ORDER BY count DESC
LIMIT 5
"""

top_connected_airports = sql_context.sql(query)

# Afficher les résultats
top_connected_airports.show()

+--------+--------+-----+
|airport1|airport2|count|
+--------+--------+-----+
|    LFBO|    LFPO|  891|
|    LEIB|    LEPA|  762|
|    EDDH|    EDDM|  636|
|    LEBL|    LEMD|  614|
|    EHAM|    LEBL|  602|
+--------+--------+-----+

